# How to generate error bars for 2Q-GST


In [ ]:
import pygsti
import pickle
import time

#If we were using MPI
# from mpi4py import MPI
# comm = MPI.COMM_WORLD
comm = None

#Load the 2-qubit results (if you don't have this file, run the 2Q-GST example)
with open("example_files/easy_2q_results.pkl","rb") as f:
    results = pickle.load(f)

#Set a memory limit
print("Memory limit was = ", results.estimates['default'].parameters.get('memLimit',"none given"))
results.estimates['default'].parameters['memLimit'] = 2.5*(1024.0)**3 # 2.5GB
print("Memory limit is now = ", results.estimates['default'].parameters['memLimit'])

In [ ]:
# error bars in reports require the presence of a fully-initialized
# "confidence region factory" within the relevant Estimate object.
# In most cases "fully-initialized" means that a Hessian has been 
# computed and projected onto the non-gauge space.
start = time.time()

# initialize a factory for the 'go0' gauge optimization within the 'default' estimate
crfact = results.estimates['default'].add_confidence_region_factory('go0', 'final')
crfact.compute_hessian(comm=comm) #optionally use multiple processors
crfact.project_hessian('intrinsic error')

end = time.time()
print("Total time=%f hours" % ((end - start) / 3600.0))

In [ ]:
#write results back to file
with open("example_files/easy_2q_results_withCI.pkl","wb") as f:
    pickle.dump(results, f)